In [17]:
from __future__ import absolute_import

%reload_ext autoreload
from igraph import Graph
import numpy as np
#import time
#from numba import njit
from scipy.sparse import coo_matrix, dok_matrix, csr_matrix
#from collections import defaultdict
#import json
import importlib

import os
os.environ['PYTHONUNBUFFERED'] = '1'

In [18]:
def ListOfBitStringsToListOfIntegers(list_of_bitstrings):
    return list(map(lambda s: int(s,2),list_of_bitstrings))
def UniformDistributionFromSupport(list_of_bitstrings):
    numvar = max(map(len,list_of_bitstrings))
    numevents = len(list_of_bitstrings)
    data = np.zeros(2 ** numvar)
    data[ListOfBitStringsToListOfIntegers(list_of_bitstrings)] = 1/numevents
    return data

In [19]:
#Some graphs and data sets to play with
InstrumentalGraph = Graph.Formula("U1->X->A->B,U2->A:B")
BiconfoundingGraph = Graph.Formula("U1->A:B,U2->A:C,A->B:C")
Evans14a = Graph.Formula("U1->A:C,U2->A:B:D,U3->B:C:D,A->B,C->D")
Evans14b = Graph.Formula("U1->A:C,U2->B:C:D,U3->A:D,A->B,B:C->D")
Evans14c = Graph.Formula("U1->A:C,U2->B:D,U3->A:D,A->B->C->D")
IceCreamGraph = Graph.Formula("U1->A,U2->B:D,U3->C:D,A->B:C,B->D")
BiconfoundingInstrumental = Graph.Formula("U1->A,U2->B:C,U3->B:D,A->B,B->C:D")
TriangleGraph = Graph.Formula("X->A,Y->A:B,Z->B:C,X->C")


#NOTE: Our inflation algorithm will always assume that the order of variables in the data set
# matches the lexocographic ordering of the non-root variables in the graph.

TriangleData=[0.12199995751046305, 0.0022969343799089472, 0.001748319476328954, 3.999015242496535e-05, 0.028907881434196828, 0.0005736087488455967, 0.0003924033706699725, 1.1247230369521505e-05, 0.0030142577390317635, 0.09234476010282468, 4.373922921480586e-05, 0.0014533921021948346, 0.0007798079722868244, 0.024091567451515063, 1.1247230369521505e-05, 0.0003849052170902915, 0.020774884184769502, 0.000396152447459813, 0.0003049249122403608, 4.998769053120669e-06, 0.10820335492385, 0.0020794879260981982, 0.0015546171755205281, 2.4993845265603346e-05, 0.0006260958239033638, 0.020273757587194154, 7.498153579681003e-06, 0.0003374169110856452, 0.0028942872817568676, 0.08976414557915113, 2.624353752888351e-05, 0.0012984302615480939, 0.002370666223442477, 4.7488306004646356e-05, 0.0999928767540993, 0.001957018084296742, 0.0006198473625869629, 8.747845842961171e-06, 0.02636975644747481, 0.0005198719815245496, 1.4996307159362007e-05, 0.000403650601039494, 0.0005498645958432735, 0.017359475229224805, 7.123245900696953e-05, 0.002346922070440154, 0.0033754188031197316, 0.10295964618712641, 0.00038740460161685187, 7.498153579681003e-06, 0.01608353942841575, 0.000306174604503641, 0.0021319750011559654, 4.248953695152569e-05, 0.09107007399427891, 0.001860791780024169, 5.998522863744803e-05, 0.0018395470115484063, 0.002570616985567304, 0.0766411271224461, 1.874538394920251e-05, 0.00048238121362614454, 0.0006410921310627258, 0.020223769896662948]
InstrumentalData=UniformDistributionFromSupport(['000','011'])
BiconfoundingData=UniformDistributionFromSupport(['000','011'])
BiconfoundingInstrumentalData=UniformDistributionFromSupport(['0000','0100','1011','1111'])
Evans14aData=UniformDistributionFromSupport(['0000','1001','1111'])
Evans14bData=UniformDistributionFromSupport(['0000','1001','1111'])
Evans14cData=UniformDistributionFromSupport(['0000','1101','1011'])
IceCreamData=UniformDistributionFromSupport(['0000','1111','1010','0011'])

In [20]:
#Set as desired.
#g=BiconfoundingGraph
#data=BiconfoundingData
#g=BiconfoundingInstrumental
#data=BiconfoundingInstrumentalData
#g = InstrumentalGraph
#data = InstrumentalData
g = Evans14b
data = Evans14bData
#g = IceCreamGraph
#data = IceCreamData


card=2
inflation_order=2
from inflation.quickgraph import QuickGraphAssessment
QuickGraphAssessment(g)

For the graph who's parental structure is given by:
['U1:U3->A', 'U2:A->B', 'U1:U2->C', 'U2:U3:B:C->D']
We utilize the following ordering of variables: [U1,U2,U3,A,B,C,D]
We identify the following screening-off relationships relevant to enforcing determinism:
Sets given as (U1s,Y,Xs) with the following meaning:
Ys are screened off from U1s by Xs.
([U1],[B],[A])
([U3],[B],[A])
([U1,U3],[B],[A])
([U1],[D],[B,C])
We identify the following screening-off non-ai expressible sets:
Sets given as (Y,Xs,Zs,U3s) with the following meaning:
Ys are screened off from Zs by Xs when U3s is different for (Y,Xs) vs Zs.
([B],[A],[C],[U2])
([B],[A],[C,D],[U2])
────────────────────────────────────────────────────────────────────────────────



In [21]:
from inflation.inflationmatrix import InflationMatrixFromGraph, NumericalAndSymbolicVectorsFromGraph
InfMat = InflationMatrixFromGraph(g, inflation_order, card, extra_expressible=True)
B_numeric, B_symbolic = NumericalAndSymbolicVectorsFromGraph(g, data, inflation_order, card, extra_expressible=True)

InfMat.shape

['A', 'B', 'C', 'D']
(['B'], ['A'], ['C'])
(['B'], ['A'], ['C', 'D'])


(160, 187680)

In [22]:
from inflation.moseklp import InfeasibilityCertificate
from inflation.mosekinfeas import InfeasibilityCertificateAUTO #Different internal LP construction, formally same LP

Sol=InfeasibilityCertificate(
    InfMat,
    B_numeric)

Inferred primal variable count: 160
Inferred dual variable count: 187680
LP constructed, initiated optimizer.
Open file 'mosek_prob.jtask'
Start writing.
done writing. Time: 0.17

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 187681          
  Cones                  : 0               
  Scalar variables       : 160             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 187681          
  Cones                  : 0               
  Scalar variables       : 160             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 6        

In [23]:
from inflation.certificate import Inequality
solveroutputcleanedup=Inequality(InfMat,
                                 B_numeric,
                                 B_symbolic,
                                 Sol)
solveroutputcleanedup.keys()


Distribution Compatibility Status: INCOMPATIBLE
Writing to file: 'inequality_output.json'


dict_keys(['Raw rolver output', 'Coefficients grouped by index', 'Coefficients grouped by symbol', 'Clean solver output'])

In [24]:
solveroutputcleanedup['Coefficients grouped by symbol']

defaultdict(list,
            {'1': ['P(0000)P(0000)',
              '2*P(0000)P(1111)',
              'P(1001)P(1001)',
              '2*P(1001)P(1111)',
              'P[A|B](1|0)P[CD|B](11|0)P[B](0)'],
             '269769104': ['2*P(0000)P(0001)'],
             '836008751': ['2*P(0000)P(0010)'],
             '832276567': ['2*P(0000)P(0011)'],
             '719629615': ['2*P(0000)P(0100)'],
             '732436353': ['2*P(0000)P(0101)'],
             '661957798': ['2*P(0000)P(0110)'],
             '658478030': ['2*P(0000)P(0111)'],
             '1358199353': ['2*P(0000)P(1000)'],
             '2': ['2*P(0000)P(1001)', 'P(1111)P(1111)'],
             '1315592493': ['2*P(0000)P(1010)'],
             '1289099476': ['2*P(0000)P(1011)'],
             '1258878076': ['2*P(0000)P(1100)'],
             '1113807566': ['2*P(0000)P(1101)'],
             '1390769665': ['2*P(0000)P(1110)'],
             '1539535823': ['P(0001)P(0001)'],
             '836325653': ['2*P(0001)P(0010)'],
            